This notebook is for transforming the full format form (xml) into the required formate for the input layer (1x455)


In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import csv
import pickle
import tqdm

In [2]:
import line_profiler
%load_ext line_profiler
%load_ext heat

In [3]:
file = 'test-race-form.xml'
datapath = r'C:\Users\Nick\Documents\GitHub\grvmodel\grv scraper\full race form'
path = os.chdir(datapath)
file_list = os.listdir()


In [8]:
#pickle Load results
resultspicklefile = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Data Processed\results-df.npy', 'rb')
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
resultsdf.head()

resultsdf.to_csv("resultsdf_r.csv")

#result_master = pd.read_csv(r'C:\Users\Nick\Documents\GitHub\grvmodel\masterfilenew.csv')
#result_master.head()

In [ ]:
#Pickle Load input layer
input_master = pickle.load("input-master.npy")

In [19]:
input_master =  input_layer_gen(file_list, resultsdf)    
with open("input-master.npy", "wb") as fp:
    pickle.dump(input_master, fp)

100%|██████████| 5128/5128 [22:21<00:00,  3.82it/s]


In [18]:
def input_layer_gen(path, result_master):
    form_master = []
    data = [resultsdf['race_id'], resultsdf['dog_id']]
    headers = ['race_id', 'dog_id']
    dogrun_df = pd.concat(data, axis=1, keys = headers)
    for i in tqdm.tqdm(file_list):
        fname, extenstion  = i.split('.')
        if extenstion != 'xml':
            continue
        root = ET.parse(i).getroot()
        track = root[0]
        #print("track =", track.text)
        date = datetime.strptime(root[1].text, "%d %b %y")
        #print("date =", date)

        full_day = []

        for race in root.iter('Race'):
            race_run = dogrun_df[dogrun_df['race_id'] == race.attrib["id"]]
            #print("race id = ",race.attrib["id"])
            #print("Distance =", race[4].text, "\n")
            #print("Grade =", race[5].text, "\n")

            #print("box", "dog id", "DID IT RUN?")
            full_race = [[[int(race[4].text[:-1])]]]
            for dog in race.iter('Dog'):
                box = dog[0].text #1
                dog_id = dog.attrib["id"]
                if not dog_id:
                    continue

                besttime = dog[2].text #2
                if not besttime.isdigit():
                    besttime = -1
                #need to edit both of these to get the numbers out
                TOT, TOT1, TOT2, TOT3 = basic_form(dog[4].text)
                TTD, TTD1, TTD2, TTD3 = basic_form(dog[5].text)

                dogprize = dog.find('DogPRIZE').text
                run = diditrun(race_run,  race.attrib["id"], dog_id)
                if run == "no":
                    continue


                forms = dog.findall('Form')
                #print(box, dog_id, besttime, TOT, TTD, dogprize, run)
                #print("\n----------\n Form\n place box dist age margin dog_time win_time split")
                form_list = [[box,besttime,TOT, TOT1, TOT2, TOT3,TTD, TTD1, TTD2, TTD3, dogprize]]
                for form in forms:
                    if not form:
                        one_form = ['-1']*8
                        form_list.append(one_form)
                        continue
                    if form[0]:
                        place = form[0].text[0]
                        if not place.isdigit():
                            place = -1
                    else:
                        place = -1

                    form_box = form[1].text
                    form_dist = form[4].text
                    form_date = datetime.strptime(form[5].text, "%d%b%y")
                    form_age = (date - form_date).days
                    #same track ???

                    if place == -1:
                        margin = -1
                    else:
                        try:
                            margin = form[8].text[:-1]
                        except:
                            margin 
                    dog_time = form[10].text
                    win_time = form[11].text
                    split = form[15].text

                    if not split:
                        split = -1

                    one_form = [place, form_box, form_dist, form_age, margin, dog_time, win_time, split]
                    form_list.append(one_form)
                    #print(place, form_box, form_dist, form_age, margin, dog_time, win_time, split)



                full_race.append(form_list)    
                #print("\n----------\n")
            flat_list = [item for sublist in full_race for item in sublist]
            #print(flat_list)
            flatter_list =  [item for sublist in flat_list for item in sublist]
            #print(flatter_list, len(flatter_list))
            #here we are going for a return object in the form [(race_id, race_form(1*409))]
            race_tuple = (race.attrib["id"], flatter_list)
            if len(flatter_list )== 409:
                form_master.append(race_tuple)
        #print(full_day)
    return(form_master)

def class_generator(form_master, results_master):
    #input
    race_ids,x = list(zip(*form_master)) 
    y = []
    #form master in format [(race_id, input_layer(1*409))]
    for i in tqdm.tqdm(range(len(race_ids))):
        #print(i)
        #print(results_master[results_master["race_id"]==int(i)])
        race_results = results_master[results_master["race_id"] == int(race_ids[i])]
        if len(race_results) != 8:
            pass
        else:
            #y.append((race_results["place_adj"].tolist(),x[i])) this is for places between 1-8
            y.append((race_results["place"].tolist(),x[i]))
        #print(race_results)
    return y

def diditrun(df, race_id, dog_id):
    #print(race_id)
    #print(dog_id)
    #print(df.size)
    if not dog_id:
        return "no"
    

    result = dog_id in df
    if result:
        return "no"
    else:
        return "yes"

def basic_form(form):
    split_1 = form.split(" ")
    split_2 = split_1[1].split("-")
    return split_2

In [6]:
classes = class_generator(form_master, result_master)

NameError: name 'form_master' is not defined

In [4]:
file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\finalnew.txt', 'rb')
data = pickle.load(file)  
classes, inputs = zip(*data)

In [12]:
print(len(inputs), len(classes))

25104 25104


In [ ]:
inputs_df = pd.DataFrame(inputs)

In [ ]:
inputs_df.head()

In [18]:
with open("final.txt", "wb") as fp:   #Pickling
    pickle.dump(classes, fp)